In [1]:
from docx import Document
from docx.shared import Pt, RGBColor, Inches, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_UNDERLINE
from docx.oxml import OxmlElement
from openai import OpenAI
import os
import sys
import nbimporter

### Get API Key

In [4]:
sys.path.append(os.path.abspath('/Users/kevinkuc/Documents/API_keys'))
from chat_api import API_KEY

NameError: name 'sys' is not defined

## Format the Run

In [3]:
def format_run(run, color=None, font_size=None, bold=False, font_name=None, underline=None, italic=False):
    """Used to format any other function that is needed"""
    
    if color:
        run.font.color.rgb = color
    if font_size:
        run.font.size = Pt(font_size)
    run.font.bold = bold
    if font_name:
        run.font.name = font_name
    if underline:
        run.font.underline = underline
    if italic:
        run.font.italic = italic


## Prompt the User Functions

In [4]:
def get_personal_info():
    name = input("Enter your full name: ").title()
    linkedin = input("Enter your LinkedIn profile link (or None): ")
    if linkedin == "none" or linkedin == "None":
        linkedin = 'www.linkedin.com'
    phone = input("Enter your phone number: ")
    email = input("Enter your email address: ")
    return name, linkedin, phone, email

def get_education_info():
    university = input("Enter the name of your school/university: ").upper()
    uni_location = input("Enter the location of your school/university (City, State): ").title()
    degree_college = input("Enter the college or school of your degree (Bachelors, Masters, Associates, High School Diploma): ").title()
    years_attended = input("Enter the years you attended the school/university (Mon Year - Mon Year): ")
    major_minor = input("Enter your major and minor: ").title()
    gpa = input("Enter your GPA (3.8/4.0): ")
    return university, uni_location, degree_college, years_attended, major_minor, gpa

def get_work_experience():
    company = input("Enter the name of the company: ").upper()
    location = input("Enter the location of the company (City, State): ").title()
    title = input("Enter your job title: ").title()
    years = input("Enter the years you worked at the company (Mon Year - Mon Year): ").title()
    client = OpenAI(api_key=API_KEY)
    completion = client.chat.completions.create(model="gpt-3.5-turbo",messages=[
    {"role": "system", "content": "You are a resume assistant, use your imagination to create 3-4 bullet points for the job that is inputted, add percentage increase and be specific"},
    {"role": "user", "content": f'{title} at {company} and worked for {years} dates'}])
    accomplishments = completion.choices[0].message.content
    return company, location, title, years, accomplishments

def get_skills(work_experiences):
    client = OpenAI(api_key=API_KEY)
    completion = client.chat.completions.create(model="gpt-3.5-turbo",messages=[
    {"role": "system", "content": "You are a resume assistant, produce what skills can be pulled from based off what I input. Please comma seperate the skills and put them all on one line"},
    {"role": "user", "content": f'{work_experiences}'}])
    skills = completion.choices[0].message.content
    return skills


## Functions for the Description

In [5]:
def add_name(doc, name):
    
    heading = doc.add_heading(name, 1)
    run = heading.runs[0]
    format_run(run, RGBColor(0,0,0), font_size=20, bold=True, font_name='Times New Roman')
    heading.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

def add_contact_info(doc, linkedin, phone, email):
    
    info = doc.add_paragraph()
    info.add_run(f'{linkedin}\t')
    info.add_run(f'\t{phone}\t')
    info.add_run(f'\t{email}')
    info.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    horizontal_line = doc.add_paragraph()
    horizontal_line.add_run("-------------------------------------------------------------------------------------------------------------------------------------------------------")
    horizontal_line.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

def education_header(doc):

    education_title = doc.add_heading("Education", level=2)
    run = education_title.runs[0]
    format_run(run, RGBColor(0,0,0), font_size=14, bold=True, font_name='Times New Roman', underline=True)
    education_title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

def add_education(doc, university, uni_location, degree_college, years_attended, major_minor, gpa):

    eduction = doc.add_paragraph()
    run1 = eduction.add_run(f"{university}, {degree_college}\t\t\t{years_attended}")
    run2 = eduction.add_run(f"\n{uni_location} - {major_minor}\t\t\t\t\t\t\t\tGPA {gpa}")
    format_run(run1, RGBColor(0,0,0), font_size=11, bold=True, font_name='Times New Roman', underline=False)
    format_run(run2, RGBColor(0,0,0), font_size=11, bold=False, font_name='Times New Roman', underline=False, italic=True)
    eduction.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

def work_header(doc):

    work_title = doc.add_heading("Work Experience", level=2)
    run = work_title.runs[0]
    format_run(run, RGBColor(0,0,0), font_size=14, bold=True, font_name='Times New Roman', underline=True)
    work_title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

def add_work(doc, company, location, title, years, accomplishments):
    
    work = doc.add_paragraph()
    run1 = work.add_run(f"{company}, {location}")
    run2 = work.add_run(f"\n{title}\t\t\t\t\t\t\t\t\t\t{years}\n")
    accomplishment_run = work.add_run(f"{accomplishments}")
    format_run(accomplishment_run, RGBColor(0, 0, 0), font_size=11, bold=False, font_name='Times New Roman', underline=False, italic=False)
    format_run(run1, RGBColor(0, 0, 0), font_size=11, bold=True, font_name='Times New Roman', underline=False)
    format_run(run2, RGBColor(0, 0, 0), font_size=11, bold=False, font_name='Times New Roman', underline=False, italic=True)
    work.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

def add_skills(doc, skills):

    skill_title = doc.add_heading("Skills", level=2)
    run = skill_title.runs[0]
    format_run(run, RGBColor(0,0,0), font_size=14, bold=True, font_name='Times New Roman', underline=True)
    skill_title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    skills_par = doc.add_paragraph()
    skills_par.add_run(f"{skills}")
    skills_par.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

## Test the Program

In [6]:
if __name__ == "__main__":
    """
    name = "kevin b. cook".upper()
    linkedin = "www.linkedin.com/in/kevin-cook"
    phone = "(908) 581-8384"
    email = "kcook234@gmail.com"
    education_info = [["university of alabama",'tuscaloosa, AL',"college of information science",'June 2016 - July 2020','mechanical engineering','3.8/4.0']]
    work_experiences = [['Publix', 'Wilmington NC', 'Cashier', 'Jan 2020 - Feb 2022', '1. Successfully processed an average of 100 transactions per hour with 99.5% accuracy, resulting in a 10% increase in checkout efficiency compared to previous cashiers.\n2. Promoted customer satisfaction by providing exceptional service, resulting in a 15% increase in positive customer feedback during my tenure.\n3. Implemented innovative cash handling techniques, resulting in a 20% decrease in cash discrepancies and enhancing overall store profitability.\n4. Demonstrated strong teamwork and collaboration skills by assisting colleagues during peak hours, contributing to a 10% decrease in customer wait times.']]
    skills = 'MySQL, Javascript, Python'
    """
    #Delete when in full production

#-----------------------------------------------------------------
  
    # Get personal information
    name, linkedin, phone, email = get_personal_info()
    
    # Get education information
    education_info = []
    add_eduction = input("Do you want to add eduction? (yes/no): ").lower()
    while add_eduction == 'yes':
        eduction = get_education_info()
        education_info.append(eduction)
        add_eduction = input("Do you want to add another education? (yes/no): ").lower()
    
    # Get work experience information (allowing for multiple experiences)
    work_experiences = []
    add_experience = input("Do you want to add work experience? (yes/no): ").lower()
    while add_experience == 'yes':
        experience = get_work_experience()
        work_experiences.append(experience)
        add_experience = input("Do you want to add another work experience? (yes/no): ").lower()

    #Get skills from work experiences
    skills = get_skills(work_experiences)
    
    
    # Create the resume template
    resume = Document()
    section = resume.sections[0]
    section.top_margin = Cm(0.5)
    section.bottom_margin = Cm(0.5)
    section.left_margin = Cm(1)
    section.right_margin = Cm(1)
    add_name(resume, name)
    add_contact_info(resume, linkedin, phone, email)
    
    education_header(resume)
    for eduction_inf in education_info:
        add_education(resume, eduction_inf[0],eduction_inf[1],eduction_inf[2],eduction_inf[3],eduction_inf[4],eduction_inf[5])

    work_header(resume)
    for work_experience in work_experiences:
        add_work(resume, work_experience[0],work_experience[1],work_experience[2],work_experience[3],work_experience[4])

    add_skills(resume, skills)

    # Save the document
    full_name = name.split()
    
    # Get the first and last words
    first_word = full_name[0] if full_name else None
    last_word = full_name[-1] if full_name else None
    
    # Join the first and last words to create a valid filename
    filename = f'{first_word}_{last_word}_resume_template.docx' if first_word and last_word else 'resume_template.docx'
    
    resume.save(filename)


Enter your full name:  Ethan Alexander
Enter your LinkedIn profile link (or None):  None
Enter your phone number:  682-252-3441
Enter your email address:  zaleeth@gmail.com
Do you want to add eduction? (yes/no):  Yes
Enter the name of your school/university:  Chisholm Trail High School
Enter the location of your school/university (City, State):  Fort Worth, Texas
Enter the college or school of your degree (Bachelors, Masters, Associates, High School Diploma):  High School Diploma
Enter the years you attended the school/university (Mon Year - Mon Year):  Aug 2020 - June 2021
Enter your major and minor:  
Enter your GPA (3.8/4.0):  3.5/4.0
Do you want to add another education? (yes/no):  No
Do you want to add work experience? (yes/no):  Yes
Enter the name of the company:  United States Marine Corps
Enter the location of the company (City, State):  Jacksonville, NC
Enter your job title:  Combat Engineer
Enter the years you worked at the company (Mon Year - Mon Year):  Nov 2021 - Jan 2024
